In [18]:
import requests
import pandas as pd
import time
from datetime import datetime
import pyodbc
from sqlalchemy import create_engine
import post_message_categorization as pmc
import comment_sentiment as cs
import ast
import os
from dotenv import load_dotenv

load_dotenv()
pd.options.display.max_rows = 50

# Replace with your Meta (Facebook) token


In [19]:
server = os.getenv('server')
username = os.getenv('db_username')
password = os.getenv('db_password')
database = os.getenv('database')

META_TOKEN = os.getenv('META_TOKEN')
API_KEY = os.getenv('API_KEY')

In [24]:
# Establish the database connection
conn_str = (f'Driver={{ODBC Driver 17 for SQL Server}};Server={server};Database={database};UID={username};PWD={password}')
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Create SQLAlchemy engine to connect to SQL Server
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn_str))

In [26]:
# Execute the query
start_tuples = cursor.execute("SELECT MAX([comments_created_time]) FROM [dbo].[tbl_facebook_comments]")

# Convert the result into a list of strings
start = [row[0] for row in start_tuples]
start[0]

'2025-01-18T14:56:38+0000'

In [5]:
# Define the date filter (e.g., comments after January 1, 2025)
SINCE_DATE = '2023-01-01T13:13:31+0000'
SINCE_TIMESTAMP = int(datetime.strptime(SINCE_DATE, "%Y-%m-%dT%H:%M:%S%z").timestamp())

# Base URL for posts
# BASE_POSTS_URL = f"https://graph.facebook.com/v20.0/466901410034470/posts?fields=message,created_time,likes.summary(true),comments.summary(true),permalink_url,id&access_token={META_TOKEN}"
BASE_POSTS_URL = f"https://graph.facebook.com/v20.0/466901410034470/posts?fields=message,created_time,likes.summary(true),comments.summary(true).filter(stream).since({SINCE_TIMESTAMP}),permalink_url,id&access_token={META_TOKEN}"

In [6]:
# Function to fetch paginated data
def fetch_page(url):
    data = []
    print("Process starting...")
    while url:
        response = requests.get(url).json()
        data.extend(response.get("data", []))
        url = response.get("paging", {}).get("next")
    print("Process completed...")
    return data

In [7]:
# Retrieve all posts
posts_data = fetch_page(BASE_POSTS_URL)

# Transform posts data into a DataFrame
posts_df = pd.json_normalize(posts_data)

posts_df.head(10)

Process starting...
Process completed...


,message,created_time,permalink_url,id,likes.data,likes.paging.cursors.before,likes.paging.cursors.after,likes.paging.next,likes.summary.total_count,likes.summary.can_like,likes.summary.has_liked,comments.data,comments.paging.cursors.before,comments.paging.cursors.after,comments.summary.order,comments.summary.total_count,comments.summary.can_comment,comments.paging.next
0,Njaanuary hustle? Start that car wash business...,2025-01-16T12:37:24+0000,https://www.facebook.com/973134274849649/posts...,466901410034470_1004121905084219,"[{'id': '8045232682161758', 'name': 'Abbie Vis...",QVFIUmdVbmhFdXNkUUR3dDZAIUmZAhNFg0RHNZAZATJORU...,QVFIUjJBNE0xMDEtVDRRRm9sa2p4cURZAd1RGUzdiRzc5V...,https://graph.facebook.com/v20.0/4669014100344...,27,True,False,"[{'created_time': '2025-01-17T12:03:37+0000', ...",WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEUxTlRjNE1qQXdN...,WTI5dGJXVnVkRjlqZAFhKemIzSTZAORFl5TWpFeU16UXdN...,chronological,2,True,NaN
1,"For crystal clear and refreshing pools, Davis ...",2025-01-14T13:20:11+0000,https://www.facebook.com/973134274849649/video...,466901410034470_1002774298552313,[],QVFIUlJHM1lwaFFRV0xYMzNnS2hqc1FYNzFGU2lxWXUxWl...,QVFIUndoRmxvYnlncFZAzcks2cXNZAYXNxSzE2M1BYTVZA...,NaN,21,True,False,[],NaN,NaN,chronological,0,True,NaN
2,Davis & Shirtliff supplied and installed a sta...,2025-01-13T12:54:13+0000,https://www.facebook.com/973134274849649/video...,466901410034470_1002078125288597,[],QVFIUlVyRE5ZAWHhCSjJfN2Q4aV9tTy1nekxhNXI2VFhub...,QVFIUlpZAZAkxLZAmdRNGpGaHZA0QmVVV1FBVktlTjJ3bm...,https://graph.facebook.com/v20.0/4669014100344...,39,True,False,"[{'created_time': '2025-01-13T13:13:31+0000', ...",WTI5dGJXVnVkRjlqZAFhKemIzSTZAPVFUwTXpnNU9UQTJN...,WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEE1TnpZAek1ETTN...,chronological,2,True,NaN
3,"Davis & Shirtliff, through its Improving Lives...",2025-01-10T09:48:30+0000,https://www.facebook.com/973134274849649/posts...,466901410034470_999921408837602,"[{'id': '8045232682161758', 'name': 'Abbie Vis...",QVFIUjlqTERZAMzNKWXVVRHZAyTWNJTC1JdndMQktqRTM3...,QVFIUlZADWkpDLUV4NjdDN2xzNm5McnltWlN1X21JUHZA2...,https://graph.facebook.com/v20.0/4669014100344...,38,True,True,"[{'created_time': '2025-01-10T14:06:13+0000', ...",WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEUyTURNeE1EUTJN...,WTI5dGJXVnVkRjlqZAFhKemIzSTZAPVEkxTXpJNE9UazJN...,chronological,2,True,NaN
4,Davis & Shirtliff has supplied and installed a...,2025-01-09T11:40:01+0000,https://www.facebook.com/973134274849649/posts...,466901410034470_999266125569797,[],QVFIUnFnZAUdtc2hFYmtKeU9qVHNfZATl1UlBoVUg3b1N2...,QVFIUlp4bXBJeXRFdUhtOXFRTUpVQjJXSU80Q19Xb2h2N0...,https://graph.facebook.com/v20.0/4669014100344...,59,True,False,"[{'created_time': '2025-01-09T12:22:59+0000', ...",WTI5dGJXVnVkRjlqZAFhKemIzSTZAOVEUyTmpRME5EWXdO...,WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEUxTVRJeE1qSTRN...,chronological,5,True,NaN
5,Davis & Shirtliff has been nominated for the K...,2025-01-07T12:35:47+0000,https://www.facebook.com/973134274849649/posts...,466901410034470_997825299047213,[],QVFIUm9BczBnRFlibS1DaVBCamhubVd2bTNGMFA1a1ljdU...,QVFIUmtIejFpS2FmbnV4NjNYSWJtS0lJQWpvMmFzR0NSNm...,https://graph.facebook.com/v20.0/4669014100344...,46,True,False,"[{'created_time': '2025-01-07T12:44:32+0000', ...",WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEkzTURZAM05UYzB...,WTI5dGJXVnVkRjlqZAFhKemIzSTZAOakl4TVRFMU56SXdN...,chronological,4,True,NaN
6,Davis & Shirtliff Tanzania recently launched a...,2025-01-03T12:25:59+0000,https://www.facebook.com/973134274849649/posts...,466901410034470_995026549327088,[],QVFIUnkxTUtlekNvNmZAXcmE0d29sb200X3ZAmRkZAnTjR...,QVFIUkJwYlFpS0tRTkl5TGNwUWNjelBPQ19xTjJjOG1VSk...,https://graph.facebook.com/v20.0/4669014100344...,45,True,False,[],NaN,NaN,chronological,0,True,NaN
7,Davis & Shirtliff wishes all our valued custom...,2024-12-25T04:00:05+0000,https://www.facebook.com/973134274849649/posts...,466901410034470_988454659984277,"[{'id': '8045232682161758', 'name': 'Abbie Vis...",QVFIUnlreUd0UHAtUm50TUNzY21vV21MLXM0RkJybVJ1eX...,QVFIUlNBcGRZATlJkNG9DcjBPbWplaUVDQzBNR2NPOXRuR...,https://graph.faceb

In [9]:
posts_df.columns

Index(['message', 'created_time', 'permalink_url', 'id', 'likes.data',
       'likes.paging.cursors.before', 'likes.paging.cursors.after',
       'likes.paging.next', 'likes.summary.total_count',
       'likes.summary.can_like', 'likes.summary.has_liked', 'comments.data',
       'comments.paging.cursors.before', 'comments.paging.cursors.after',
       'comments.summary.order', 'comments.summary.total_count',
       'comments.summary.can_comment', 'comments.paging.next'],
      dtype='object')

In [10]:
# df = posts_df.loc[:, ['message', 'created_time', 'likes.summary.total_count', 'permalink_url', 'id']]
# df.rename(columns={"likes.summary.total_count": "likes_summary_total_count"}, inplace=True)
# df['exists'] = 0
# df

posts_df = posts_df.loc[:, ['message', 'created_time', 'likes.summary.total_count', 'permalink_url', 'id', 'comments.data', 'comments.summary.total_count']]
posts_df.rename(columns={"likes.summary.total_count": "likes_summary_total_count","comments.data": "comments_data","comments.summary.total_count": "comments_summary_total_count"}, inplace=True)
posts_df['exists'] = 0
posts_df.head(5)

,message,created_time,likes_summary_total_count,permalink_url,id,comments_data,comments_summary_total_count,exists
0,Njaanuary hustle? Start that car wash business...,2025-01-16T12:37:24+0000,27,https://www.facebook.com/973134274849649/posts...,466901410034470_1004121905084219,"[{'created_time': '2025-01-17T12:03:37+0000', ...",2,0
1,"For crystal clear and refreshing pools, Davis ...",2025-01-14T13:20:11+0000,21,https://www.facebook.com/973134274849649/video...,466901410034470_1002774298552313,[],0,0
2,Davis & Shirtliff supplied and installed a sta...,2025-01-13T12:54:13+0000,39,https://www.facebook.com/973134274849649/video...,466901410034470_1002078125288597,"[{'created_time': '2025-01-13T13:13:31+0000', ...",2,0
3,"Davis & Shirtliff, through its Improving Lives...",2025-01-10T09:48:30+0000,38,https://www.facebook.com/973134274849649/posts...,466901410034470_999921408837602,"[{'created_time': '2025-01-10T14:06:13+0000', ...",2,0
4,Davis & Shirtliff has supplied and installed a...,2025-01-09T11:40:01+0000,59,https://www.facebook.com/973134274849649/posts...,466901410034470_999266125569797,"[{'created_time': '2025-01-09T12:22:59+0000', ...",5,0


In [27]:
def get_ids(query):
    # Execute the query
    post_id_tuples = cursor.execute(query)

    # Convert the result into a list of strings
    post_id = [row[0] for row in post_id_tuples]
    return post_id

In [28]:
# Execute the query
post_id = get_ids("SELECT [post_id] FROM [dbo].[tbl_facebook_posts]")
post_id


['466901410034470_1004121905084219',
 '466901410034470_1002774298552313',
 '466901410034470_1002078125288597',
 '466901410034470_999921408837602',
 '466901410034470_999266125569797',
 '466901410034470_984204927075917',
 '466901410034470_983333627163047',
 '466901410034470_982797703883306',
 '466901410034470_979901360839607',
 '466901410034470_974396208056789',
 '466901410034470_972295164933560',
 '466901410034470_972293291600414',
 '466901410034470_970319718464438',
 '466901410034470_957824866380590',
 '466901410034470_964642149032195',
 '466901410034470_964058292423914',
 '466901410034470_963374495825627',
 '466901410034470_962681582561585',
 '466901410034470_952574186905658',
 '466901410034470_950459343783809',
 '466901410034470_949842670512143',
 '466901410034470_949682323861511',
 '466901410034470_944550934374650',
 '466901410034470_943828071113603',
 '466901410034470_943224024507341',
 '466901410034470_941662947996782',
 '466901410034470_940377604791983',
 '466901410034470_9381358

In [13]:
# Insert the posts dataframe into SQL Server
for index, row in posts_df.iterrows():
     if row.id in post_id:
          posts_df['exists'] = 1
     else:
          continue

In [14]:
categorizer = pmc.ContentCategorizer(api_key=API_KEY)
    
categorized_df = categorizer.batch_categorization(posts_df[posts_df['exists'] == 0],post_column = "message")
categorized_df

,message,created_time,likes_summary_total_count,permalink_url,id,comments_data,comments_summary_total_count,exists,Primary_Category,Secondary_Categories,Confidence_Score,Keywords,Categorization_Reasoning
0,Njaanuary hustle? Start that car wash business...,2025-01-16T12:37:24+0000,27,https://www.facebook.com/973134274849649/posts...,466901410034470_1004121905084219,"[{'created_time': '2025-01-17T12:03:37+0000', ...",2,0,Products,,0.90,"Pressure Washers, quality, affordability, solu...",The post highlights the features and benefits ...
1,"For crystal clear and refreshing pools, Davis ...",2025-01-14T13:20:11+0000,21,https://www.facebook.com/973134274849649/video...,466901410034470_1002774298552313,[],0,0,Products,,0.90,"swimming pool equipment, high-quality, crystal...",The post highlights specific swimming pool equ...
2,Davis & Shirtliff supplied and installed a sta...,2025-01-13T12:54:13+0000,39,https://www.facebook.com/973134274849649/video...,466901410034470_1002078125288597,"[{'created_time': '2025-01-13T13:13:31+0000', ...",2,0,Sustainability,,0.95,"solar-powered, borehole pumping system, sustai...",The post highlights the installation of a sola...
3,"Davis & Shirtliff, through its Improving Lives...",2025-01-10T09:48:30+0000,38,https://www.facebook.com/973134274849649/posts...,466901410034470_999921408837602,"[{'created_time': '2025-01-10T14:06:13+0000', ...",2,0,Sustainability,Events,0.90,"solar pumping equipment, sustainable water sol...",The post primarily focuses on the donation and...
4,Davis & Shirtliff has supplied and installed a...,2025-01-09T11:40:01+0000,59,https://www.facebook.com/973134274849649/posts...,466901410034470_999266125569797,"[{'created_time': '2025-01-09T12:22:59+0000', ...",5,0,Sustainability,Products,0.90,"Hybrid Solar Pumping System, sustainable energ...",The post emphasizes the installation of a Hybr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,D&S Mechanical Workshop staff visited the asse...,2013-05-10T12:50:35+0000,1,https://www.facebook.com/DavisandShirtliff/pho...,466901410034470_525358744188736,[],0,0,Events,,0.85,"D&S Mechanical Workshop, assembly plant, Gener...",The post describes a visit by D&S Mechanical W...
1996,Inauguration of the recently completed Distirb...,2013-04-30T14:02:03+0000,1,https://www.facebook.com/DavisandShirtliff/pho...,466901410034470_521250344599576,[],0,0,Events,,0.90,"inauguration, Distribution Centre, completed",The post discusses the inauguration of a newly...
1997,NaN,2013-01-18T14:25:44+0000,6,https://www.facebook.com/466901410034470/posts...,466901410034470_395665952722975,[],0,0,Uncategorized,None,0.50,None,
1998,NaN,2013-01-18T13:41:34+0000,3,https://www.facebook.com/466901410034470/posts...,466901410034470_781545676426712,[],0,0,Uncategorized,None,0.50,None,


In [16]:
categorized_df.shape

(2000, 13)

In [19]:
categorized_df.columns

Index(['message', 'created_time', 'likes_summary_total_count', 'permalink_url',
       'id', 'comments_data', 'comments_summary_total_count', 'exists',
       'Primary_Category', 'Secondary_Categories', 'Confidence_Score',
       'Keywords', 'Categorization_Reasoning'],
      dtype='object')

In [17]:
def get_post_insights(post_id, access_token):
    insights_url = f"https://graph.facebook.com/v20.0/{post_id}/insights"
    params = {
        "metric": "post_impressions,post_impressions_unique",
        "access_token": access_token
    }

    try:
        # Fetch insights data
        response = requests.get(insights_url, params=params)
        response.raise_for_status()
        insights_data = response.json().get("data", [])
        
        # Process data into a DataFrame
        insights_list = []
        for insight in insights_data:
            insight_name = insight.get("name")
            for value_item in insight.get("values", []):
                insights_list.append({
                    "InsightName": insight_name,
                    "InsightValue": value_item.get("value")
                })
        
        # Convert to DataFrame
        insights_df = pd.DataFrame(insights_list)
        return insights_df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching insights: {e}")
        return pd.DataFrame()



In [20]:
categorized_df['post_impressions'] = 0
categorized_df['post_impressions_unique'] = 0

In [23]:
categorized_df.loc[categorized_df['id'] == row.id, 'post_impressions_unique']

In [24]:
categorized_df.loc[categorized_df['id'] == row.id]

,message,created_time,likes_summary_total_count,permalink_url,id,comments_data,comments_summary_total_count,exists,Primary_Category,Secondary_Categories,Confidence_Score,Keywords,Categorization_Reasoning,post_impressions,post_impressions_unique
1,"For crystal clear and refreshing pools, Davis ...",2025-01-14T13:20:11+0000,21,https://www.facebook.com/973134274849649/video...,466901410034470_1002774298552313,[],0,0,Products,,0.9,"swimming pool equipment, high-quality, crystal...",The post highlights specific swimming pool equ...,57,None


In [25]:
# Get post insights
for index, row in categorized_df.iterrows():
     if row.id in post_id:
         continue
        #   posts_df['exists'] = 1
     else:
        insights_df = get_post_insights(row.id, META_TOKEN)
        transformed_df = insights_df.pivot(columns="InsightName", values="InsightValue")
        categorized_df.loc[categorized_df['id'] == row.id, 'post_impressions'] = transformed_df[["post_impressions", "post_impressions_unique"]].max().to_frame().T['post_impressions'][0]
        categorized_df.loc[categorized_df['id'] == row.id, 'post_impressions_unique'] = transformed_df[["post_impressions", "post_impressions_unique"]].max().to_frame().T['post_impressions_unique'][0]


In [12]:

categorized_df.rename(columns={"message": "post_message","created_time": "post_created_time","likes_summary_total_count": "post_like_count","permalink_url": "post_url","id": "post_id"}, inplace=True)
df2 = categorized_df[['post_message', 'post_created_time', 'post_like_count', 'post_url', 'post_id', 'post_impressions', 'post_impressions_unique', 'Primary_Category', 'Secondary_Categories', 'Confidence_Score', 'Keywords', 'Categorization_Reasoning']]
df2.to_sql('tbl_facebook_posts', engine, if_exists='append', index=False)

del df2

In [10]:
categorized_df = pd.read_csv('categorized_df.csv')

In [13]:
categorized_df.columns

Index(['post_message', 'post_created_time', 'post_like_count', 'post_url',
       'post_id', 'comments_data', 'comments_summary_total_count', 'exists',
       'Primary_Category', 'Secondary_Categories', 'Confidence_Score',
       'Keywords', 'Categorization_Reasoning', 'post_impressions',
       'post_impressions_unique'],
      dtype='object')

In [90]:
df_comments = categorized_df.loc[:, ['post_id','comments_data']]
# df_comments.rename(columns={"id": "post_id"}, inplace=True)
df_comments = df_comments[df_comments['comments_data'] != '[]']
df_comments.head(10)

,post_id,comments_data
0,466901410034470_1004121905084219,"[{'created_time': '2025-01-17T12:03:37+0000', ..."
2,466901410034470_1002078125288597,"[{'created_time': '2025-01-13T13:13:31+0000', ..."
3,466901410034470_999921408837602,"[{'created_time': '2025-01-10T14:06:13+0000', ..."
4,466901410034470_999266125569797,"[{'created_time': '2025-01-09T12:22:59+0000', ..."
5,466901410034470_997825299047213,"[{'created_time': '2025-01-07T12:44:32+0000', ..."
7,466901410034470_988454659984277,"[{'created_time': '2024-12-25T05:33:21+0000', ..."
8,466901410034470_987153653447711,"[{'created_time': '2024-12-31T18:10:24+0000', ..."
9,466901410034470_984204927075917,"[{'created_time': '2024-12-19T09:41:40+0000', ..."
10,466901410034470_983333627163047,"[{'created_time': '2024-12-18T07:42:55+0000', ..."
11,466901410034470_982797703883306,"[{'created_time': '2024-12-17T13:58:03+0000', ..."


In [91]:
df_comments['comments_data'] = df_comments['comments_data'].apply(ast.literal_eval)

df_comments['comments_data']


0       [{'created_time': '2025-01-17T12:03:37+0000', ...
2       [{'created_time': '2025-01-13T13:13:31+0000', ...
3       [{'created_time': '2025-01-10T14:06:13+0000', ...
4       [{'created_time': '2025-01-09T12:22:59+0000', ...
5       [{'created_time': '2025-01-07T12:44:32+0000', ...
7       [{'created_time': '2024-12-25T05:33:21+0000', ...
8       [{'created_time': '2024-12-31T18:10:24+0000', ...
9       [{'created_time': '2024-12-19T09:41:40+0000', ...
10      [{'created_time': '2024-12-18T07:42:55+0000', ...
11      [{'created_time': '2024-12-17T13:58:03+0000', ...
12      [{'created_time': '2024-12-13T08:33:37+0000', ...
14      [{'created_time': '2024-12-11T10:44:51+0000', ...
15      [{'created_time': '2024-12-10T09:09:03+0000', ...
17      [{'created_time': '2024-12-05T13:50:49+0000', ...
18      [{'created_time': '2024-12-05T09:48:36+0000', ...
20      [{'created_time': '2024-12-02T07:31:59+0000', ...
21      [{'created_time': '2024-11-29T13:06:55+0000', ...
22      [{'cre

In [94]:
df_comments_flattened = pd.DataFrame()
for index, row in df_comments.iterrows():
    df8 = pd.DataFrame(row.comments_data)
    df8['post_id'] = row.post_id
    df_comments_flattened = pd.concat([df_comments_flattened, df8],ignore_index=True)
print(df_comments_flattened)

                  created_time  \
0     2025-01-17T12:03:37+0000   
1     2025-01-18T10:44:22+0000   
2     2025-01-13T13:13:31+0000   
3     2025-01-18T14:56:38+0000   
4     2025-01-10T14:06:13+0000   
...                        ...   
2979  2024-12-21T14:54:37+0000   
2980  2024-12-21T14:55:22+0000   
2981  2024-12-27T06:06:34+0000   
2982  2023-06-03T08:41:00+0000   
2983  2023-07-18T14:42:27+0000   

                                                message  \
0                                                         
1     Ebu price for Water proof cables ya solar powe...   
2     Waw Wonder full job to Davis and shirtliff I c...   
3     How much does it cost for pump,  on a height o...   
4                                        Doing good job   
...                                                 ...   
2979  Can I get quotation for house solar panel inst...   
2980                               And all requirements   
2981  Lucy Kamau Thank you for the inquiry Lucy, ple...   
2

In [96]:
df_comments_flattened.rename(columns={"created_time": "comments_created_time","message": "comments_message","id": "comments_id"}, inplace=True)
df_comments_flattened = df_comments_flattened[['comments_created_time', 'comments_message', 'comments_id', 'post_id']]
df_comments_flattened = df_comments_flattened[df_comments_flattened['comments_message'] != '']
print(df_comments_flattened.shape)
df_comments_flattened.head(10)

(2954, 4)


,comments_created_time,comments_message,comments_id,post_id
1,2025-01-18T10:44:22+0000,Ebu price for Water proof cables ya solar powe...,1004121905084219_462212340285355,466901410034470_1004121905084219
2,2025-01-13T13:13:31+0000,Waw Wonder full job to Davis and shirtliff I c...,1002078125288597_954389906136758,466901410034470_1002078125288597
3,2025-01-18T14:56:38+0000,"How much does it cost for pump, on a height o...",1002078125288597_1097630378764743,466901410034470_1002078125288597
4,2025-01-10T14:06:13+0000,Doing good job,999921408837602_1160310462102511,466901410034470_999921408837602
5,2025-01-10T22:28:53+0000,Great job keep up guys 👏👏👏👏,999921408837602_925328996380732,466901410034470_999921408837602
6,2025-01-09T12:22:59+0000,Make sure these week Nita visit branches yenu ...,999266125569797_516644460771523,466901410034470_999266125569797
7,2025-01-09T17:20:20+0000,How much,999266125569797_611391394693029,466901410034470_999266125569797
8,2025-01-13T06:10:13+0000,Edwin Tigers Most welcome.\n^AV,999266125569797_1647988492767259,466901410034470_999266125569797
9,2025-01-13T06:12:50+0000,Muiyuro Metumi The cost of solar solutions var...,999266125569797_1648966539162475,466901410034470_999266125569797
10,2025-01-13T06:33:07+0000,Davis & Shirtliff Nisha cm thanks.,999266125569797_1151212283010340,466901410034470_999266125569797


In [30]:
Comment_id = get_ids("SELECT [comments_id] FROM [dbo].[tbl_facebook_comments]")
Comment_id

ProgrammingError: The cursor's connection has been closed.

In [ ]:
# Check if the post exists
df_comments_flattened['exists'] = 0
for index, row in df_comments_flattened.iterrows():
     if row.comments_id in Comment_id:
          df_comments_flattened['exists'] = 1
     else:
          continue
     
# Categorize posts
categorizer = pmc.ContentCategorizer(api_key=API_KEY)
categorized_df = categorizer.batch_categorization(posts_df[posts_df['exists'] == 0],post_column = "message")

In [98]:
# Check if the post exists
df_comments_flattened['exists'] = 0
for index, row in df_comments_flattened.iterrows():
     if row.comments_id in Comment_id:
          df_comments_flattened['exists'] = 1
     else:
          continue
     
# Perform batch sentiment analysis
analyzer = cs.SentimentAnalyzer(api_key=API_KEY)
processed_df = analyzer.batch_sentiment_analysis(df_comments_flattened[df_comments_flattened['exists'] == 0], comment_column='comments_message')


,comments_created_time,comments_message,comments_id,post_id,Sentiment,Confidence_Score,Key_Emotions,Reasoning
1,2025-01-18T10:44:22+0000,Ebu price for Water proof cables ya solar powe...,1004121905084219_462212340285355,466901410034470_1004121905084219,Neutral,0.70,,The comment is a straightforward inquiry about...
2,2025-01-13T13:13:31+0000,Waw Wonder full job to Davis and shirtliff I c...,1002078125288597_954389906136758,466901410034470_1002078125288597,Positive,0.95,"joy, satisfaction, pride",The comment expresses a positive sentiment tow...
3,2025-01-18T14:56:38+0000,"How much does it cost for pump, on a height o...",1002078125288597_1097630378764743,466901410034470_1002078125288597,Neutral,0.85,"curiosity, inquisitiveness",The comment is a straightforward inquiry about...
4,2025-01-10T14:06:13+0000,Doing good job,999921408837602_1160310462102511,466901410034470_999921408837602,Positive,0.85,"satisfaction, appreciation",The comment 'Doing good job' expresses a posit...
5,2025-01-10T22:28:53+0000,Great job keep up guys 👏👏👏👏,999921408837602_925328996380732,466901410034470_999921408837602,Positive,0.95,"Joy, Pride, Encouragement",The comment expresses a positive sentiment thr...
...,...,...,...,...,...,...,...,...
2979,2024-12-21T14:54:37+0000,Can I get quotation for house solar panel inst...,781536331904308_1132945778200870,466901410034470_781536331904308,Neutral,0.85,"curiosity, inquisitiveness",The comment expresses a request for informatio...
2980,2024-12-21T14:55:22+0000,And all requirements,781536331904308_957062179672735,466901410034470_781536331904308,Neutral,0.70,,The comment 'And all requirements' is vague an...
2981,2024-12-27T06:06:34+0000,"Lucy Kamau Thank you for the inquiry Lucy, ple...",781536331904308_566713862807907,466901410034470_781536331904308,Neutral,0.85,"gratitude, engagement","The comment expresses a neutral tone, primaril..."
2982,2023-06-03T08:41:00+0000,"How can you help this old man, I always find h...",562961250428485_3564458427210185,466901410034470_562961323761811,Neutral,0.85,"Concern, Empathy",The comment expresses a concern for an individ...


In [102]:
processed_df.to_sql('tbl_facebook_comments', engine, if_exists='append', index=False)

72

In [105]:
comment_id = get_ids("SELECT [comment_id] FROM [dbo].[tbl_facebook_comments]")
comment_id

['1002078125288597_954389906136758',
 '999921408837602_1160310462102511',
 '999921408837602_925328996380732',
 '999266125569797_516644460771523',
 '999266125569797_611391394693029',
 '999266125569797_1647988492767259',
 '999266125569797_1648966539162475',
 '999266125569797_1151212283010340']

In [106]:
# Insert the comments dataframe into SQL Server
for index, row in expanded_df.iterrows():
     if row.id in comment_id:
          continue
     else:
          cursor.execute("INSERT INTO [dbo].[tbl_facebook_comments] ([comment], [comment_created_time], [comment_id]) values(?,?,?)", row.message, row.created_time, row.id)
print('Data inserted to DB') 


Data inserted to DB


In [31]:
conn.commit()
conn.close()

ProgrammingError: Attempt to use a closed connection.